In [8]:
!pip install torch_geometric
!pip install matplotlib
!pip install rdkit
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.1+cu117.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.1+cu117.html


# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt


def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

import os.path as osp

import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import BatchNorm, PNAConv, global_add_pool
from torch_geometric.utils import degree

2.5.0+cu121
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu117.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu117.html


Loading the data from the CSV and preprocessing it by removing the Hydrogen atoms, converting SMILES to Graph and one-hot encoding the Atom Type.

In [2]:
import torch
import pandas as pd
from rdkit import Chem
from torch_geometric.data import Data

def edge_index_to_adj(edge_index, num_nodes):
    # Create an empty adjacency matrix
    adj = torch.zeros(num_nodes, num_nodes)

    # Fill the adjacency matrix using the edge indices
    adj[edge_index[0], edge_index[1]] = 1
    adj[edge_index[1], edge_index[0]] = 1  # For undirected graphs, if edge (i,j) exists, edge (j,i) also exists

    return adj


def load_qm9_smiles(csv_file):
    # Read the CSV file containing the QM9 dataset
    df = pd.read_csv(csv_file)

    # Extract SMILES strings
    smiles_list = df['smiles'].tolist()

    return smiles_list


csv_file = "qm9.csv"  # Replace with the path to your QM9 CSV file
qm9_smiles = load_qm9_smiles(csv_file)

def remove_hydrogen_from_smiles(smiles_list):
    modified_smiles = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print("Invalid SMILES:", smiles)
            continue
        mol = Chem.RemoveHs(mol)
        modified_smiles.append(Chem.MolToSmiles(mol))
    return modified_smiles


modified_smiles = remove_hydrogen_from_smiles(qm9_smiles)
def smiles_to_graph(smiles):
    # Parse the SMILES string
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None

    # Get node features (atomic numbers)
    atomic_numbers = [atom.GetAtomicNum() for atom in mol.GetAtoms()]

    # Get edge indices (connectivity)
    edge_index = []
    for bond in mol.GetBonds():
        start_idx = bond.GetBeginAtomIdx()
        end_idx = bond.GetEndAtomIdx()
        edge_index.append([start_idx, end_idx])

    # Convert edge indices to PyTorch tensor
    edge_index = torch.tensor(edge_index).t().contiguous()

    # Convert node features to PyTorch tensor
    node_features = torch.tensor(atomic_numbers, dtype=torch.float).unsqueeze(1)

    return node_features, edge_index


filtered_dataset = []

# Define encoding mappings
encoding_mappings = {
    7: [0, 0, 1, 0, 0],
    8: [0, 0, 0, 1, 0],
    6: [0, 1, 0, 0, 0],
    9: [0, 0, 0, 0, 1]
}

# Iterate over modified SMILES
for smile in modified_smiles:
    try:
        # Convert SMILES to graph representation
        node_features, edge_index1 = smiles_to_graph(smile)


        num_nodes = node_features.shape[0]
        if num_nodes > 1:
            # Convert node features to one-hot encoding
            one_hot_encoded = torch.tensor([encoding_mappings[num.item()] for num in node_features], dtype=torch.float32)

            # Create Data object and add it to the filtered dataset
            graph = Data(x=one_hot_encoded, edge_index=edge_index1, num_nodes=num_nodes)
            filtered_dataset.append(graph)
    except Exception as e:
        print(f"Error processing SMILES: {smile}. {e}")

Defining the modules need for GNN

In [3]:
from torch import nn
import torch
import math

def unsorted_segment_sum(data, segment_ids, num_segments, normalization_factor, aggregation_method: str):
    """Custom PyTorch op to replicate TensorFlow's `unsorted_segment_sum`.
        Normalization: 'sum' or 'mean'.
    """
    result_shape = (num_segments, data.size(1))
    result = data.new_full(result_shape, 0)  # Init empty result tensor.
    segment_ids = segment_ids.unsqueeze(-1).expand(-1, data.size(1))
    result.scatter_add_(0, segment_ids, data)
    if aggregation_method == 'sum':
        result = result / normalization_factor

    if aggregation_method == 'mean':
        norm = data.new_zeros(result.shape)
        norm.scatter_add_(0, segment_ids, data.new_ones(data.shape))
        norm[norm == 0] = 1
        result = result / norm
    return result

class GCL(nn.Module):
    def __init__(self, input_nf, output_nf, hidden_nf, normalization_factor, aggregation_method,
                 edges_in_d=0, nodes_att_dim=0, act_fn=nn.SiLU(), attention=False):
        super(GCL, self).__init__()
        input_edge = input_nf * 2
        self.normalization_factor = normalization_factor
        self.aggregation_method = aggregation_method
        self.attention = attention

        self.edge_mlp = nn.Sequential(
            nn.Linear(input_edge + edges_in_d, hidden_nf),
            act_fn,
            nn.Linear(hidden_nf, hidden_nf),
            act_fn)

        self.node_mlp = nn.Sequential(
            nn.Linear(hidden_nf + input_nf + nodes_att_dim, hidden_nf),
            act_fn,
            nn.Linear(hidden_nf, output_nf))

        if self.attention:
            self.att_mlp = nn.Sequential(
                nn.Linear(hidden_nf, 1),
                nn.Sigmoid())

    def edge_model(self, source, target, edge_attr, edge_mask):
        if edge_attr is None:  # Unused.
            out = torch.cat([source, target], dim=1)
        else:
            out = torch.cat([source, target, edge_attr], dim=1)

        mij = self.edge_mlp(out)

        if self.attention:
            att_val = self.att_mlp(mij)
            out = mij * att_val
        else:
            out = mij

        if edge_mask is not None:
            out = out * edge_mask
        return out, mij

    def node_model(self, x, edge_index, edge_attr, node_attr):
        row, col = edge_index
        agg = unsorted_segment_sum(edge_attr, row, num_segments=x.size(0),
                                   normalization_factor=self.normalization_factor,
                                   aggregation_method=self.aggregation_method)
        if node_attr is not None:
            agg = torch.cat([x, agg, node_attr], dim=1)
        else:
            agg = torch.cat([x, agg], dim=1)
        out = x + self.node_mlp(agg)
        return out, agg

    def forward(self, h, edge_index, edge_attr=None, node_attr=None, node_mask=None, edge_mask=None):
        row, col = edge_index
        edge_feat, mij = self.edge_model(h[row], h[col], edge_attr, edge_mask)
        h, agg = self.node_model(h, edge_index, edge_feat, node_attr)
        if node_mask is not None:
            h = h * node_mask
        return h, mij

class GNN(nn.Module):
    def __init__(self, in_node_nf, in_edge_nf, hidden_nf, out_node_nf,aggregation_method='sum', device='cpu',
                 act_fn=nn.SiLU(), n_layers=4, attention=False,
                 normalization_factor=100, ):
        super(GNN, self).__init__()
        if out_node_nf is None:
            out_node_nf = in_node_nf
        self.hidden_nf = hidden_nf
        self.device = device
        self.n_layers = n_layers
        ### Encoder
        self.embedding = nn.Linear(in_node_nf, self.hidden_nf)
        self.embedding_out = nn.Linear(self.hidden_nf, out_node_nf)
        for i in range(0, n_layers):
            self.add_module("gcl_%d" % i, GCL(
                self.hidden_nf, self.hidden_nf, self.hidden_nf,
                normalization_factor=normalization_factor,
                aggregation_method=aggregation_method,
                edges_in_d=in_edge_nf, act_fn=act_fn,
                attention=attention))
        self.to(self.device)

    def forward(self, h, edges, edge_attr=None, node_mask=None, edge_mask=None):
        # Edit Emiel: Remove velocity as input
        h = self.embedding(h)
        for i in range(0, self.n_layers):
            h, _ = self._modules["gcl_%d" % i](h, edges, edge_attr=edge_attr, node_mask=node_mask, edge_mask=edge_mask)
        h = self.embedding_out(h)

        # Important, the bias of the last linear might be non-zero
        if node_mask is not None:
            h = h * node_mask
        return h
import numpy as np

def fully_connected_graph(num_nodes):
    # Generate all possible pairs of nodes
    nodes = np.arange(num_nodes)
    pairs = np.array(np.meshgrid(nodes, nodes)).T.reshape(-1, 2)

    # Filter out self-loops (optional, depending on your requirements)
    pairs = pairs[pairs[:, 0] != pairs[:, 1]]

    # Create the edge index tensor
    edge_index = torch.tensor(pairs, dtype=torch.long).t().contiguous()

    return edge_index





Defining the modules needed for EGNN


In [4]:

def coord2diff(x, edge_index, norm_constant=1):
    row, col = edge_index
    coord_diff = x[row] - x[col]
    radial = torch.sum((coord_diff) ** 2, 1).unsqueeze(1)
    norm = torch.sqrt(radial + 1e-8)
    coord_diff = coord_diff/(norm + norm_constant)
    return radial, coord_diff
class GCL(nn.Module):
    def __init__(self, input_nf, output_nf, hidden_nf, normalization_factor, aggregation_method,
                 edges_in_d=0, nodes_att_dim=0, act_fn=nn.SiLU(), attention=False):
        super(GCL, self).__init__()
        input_edge = input_nf * 2
        self.normalization_factor = normalization_factor
        self.aggregation_method = aggregation_method
        self.attention = attention

        self.edge_mlp = nn.Sequential(
            nn.Linear(input_edge + edges_in_d, hidden_nf),
            act_fn,
            nn.Linear(hidden_nf, hidden_nf),
            act_fn)

        self.node_mlp = nn.Sequential(
            nn.Linear(hidden_nf + input_nf + nodes_att_dim, hidden_nf),
            act_fn,
            nn.Linear(hidden_nf, output_nf))

        if self.attention:
            self.att_mlp = nn.Sequential(
                nn.Linear(hidden_nf, 1),
                nn.Sigmoid())

    def edge_model(self, source, target, edge_attr, edge_mask):
        if edge_attr is None:  # Unused.
            out = torch.cat([source, target], dim=1)
        else:
            out = torch.cat([source, target, edge_attr], dim=1)
        mij = self.edge_mlp(out)

        if self.attention:
            att_val = self.att_mlp(mij)
            out = mij * att_val
        else:
            out = mij

        if edge_mask is not None:
            out = out * edge_mask
        return out, mij

    def node_model(self, x, edge_index, edge_attr, node_attr):
        row, col = edge_index
        agg = unsorted_segment_sum(edge_attr, row, num_segments=x.size(0),
                                   normalization_factor=self.normalization_factor,
                                   aggregation_method=self.aggregation_method)
        if node_attr is not None:
            agg = torch.cat([x, agg, node_attr], dim=1)
        else:
            agg = torch.cat([x, agg], dim=1)
        out = x + self.node_mlp(agg)
        return out, agg

    def forward(self, h, edge_index, edge_attr=None, node_attr=None, node_mask=None, edge_mask=None):
        row, col = edge_index
        edge_feat, mij = self.edge_model(h[row], h[col], edge_attr, edge_mask)
        h, agg = self.node_model(h, edge_index, edge_feat, node_attr)
        if node_mask is not None:
            h = h * node_mask
        return h, mij


class EquivariantUpdate(nn.Module):
    def __init__(self, hidden_nf, normalization_factor, aggregation_method,
                 edges_in_d=1, act_fn=nn.SiLU(), tanh=False, coords_range=10.0):
        super(EquivariantUpdate, self).__init__()
        self.tanh = tanh
        self.coords_range = coords_range
        input_edge = hidden_nf * 2 + edges_in_d
        layer = nn.Linear(hidden_nf, 1, bias=False)
        torch.nn.init.xavier_uniform_(layer.weight, gain=0.001)
        self.coord_mlp = nn.Sequential(
            nn.Linear(input_edge, hidden_nf),
            act_fn,
            nn.Linear(hidden_nf, hidden_nf),
            act_fn,
            layer)
        self.normalization_factor = normalization_factor
        self.aggregation_method = aggregation_method

    def coord_model(self, h, coord, edge_index, coord_diff, edge_attr, edge_mask):
        row, col = edge_index
        input_tensor = torch.cat([h[row], h[col], edge_attr], dim=1)
        if self.tanh:
            trans = coord_diff * torch.tanh(self.coord_mlp(input_tensor)) * self.coords_range
        else:
            trans = coord_diff * self.coord_mlp(input_tensor)
        if edge_mask is not None:
            trans = trans * edge_mask
        agg = unsorted_segment_sum(trans, row, num_segments=coord.size(0),
                                   normalization_factor=self.normalization_factor,
                                   aggregation_method=self.aggregation_method)
        coord = coord + agg
        return coord

    def forward(self, h, coord, edge_index, coord_diff, edge_attr=None, node_mask=None, edge_mask=None):
        coord = self.coord_model(h, coord, edge_index, coord_diff, edge_attr, edge_mask)
        if node_mask is not None:
            coord = coord * node_mask
        return coord


class EquivariantBlock(nn.Module):
    def __init__(self, hidden_nf, edge_feat_nf=2, device='cpu', act_fn=nn.SiLU(), n_layers=2, attention=True,
                 norm_diff=True, tanh=False, coords_range=15, norm_constant=1, sin_embedding=None,
                 normalization_factor=100, aggregation_method='sum'):
        super(EquivariantBlock, self).__init__()
        self.hidden_nf = hidden_nf
        self.device = device
        self.n_layers = n_layers
        self.coords_range_layer = float(coords_range)
        self.norm_diff = norm_diff
        self.norm_constant = norm_constant
        self.sin_embedding = sin_embedding
        self.normalization_factor = normalization_factor
        self.aggregation_method = aggregation_method

        for i in range(0, n_layers):
            self.add_module("gcl_%d" % i, GCL(self.hidden_nf, self.hidden_nf, self.hidden_nf, edges_in_d=edge_feat_nf,
                                              act_fn=act_fn, attention=attention,
                                              normalization_factor=self.normalization_factor,
                                              aggregation_method=self.aggregation_method))
        self.add_module("gcl_equiv", EquivariantUpdate(hidden_nf, edges_in_d=edge_feat_nf, act_fn=nn.SiLU(), tanh=tanh,
                                                       coords_range=self.coords_range_layer,
                                                       normalization_factor=self.normalization_factor,
                                                       aggregation_method=self.aggregation_method))
        self.to(self.device)

    def forward(self, h, x, edge_index, node_mask=None, edge_mask=None, edge_attr=None):
        # Edit Emiel: Remove velocity as input
        distances, coord_diff = coord2diff(x, edge_index, self.norm_constant)
        if self.sin_embedding is not None:
            distances = self.sin_embedding(distances)
        edge_attr = torch.cat([distances, edge_attr], dim=1)
        for i in range(0, self.n_layers):
            h, _ = self._modules["gcl_%d" % i](h, edge_index, edge_attr=edge_attr, node_mask=node_mask, edge_mask=edge_mask)
        x = self._modules["gcl_equiv"](h, x, edge_index, coord_diff, edge_attr, node_mask, edge_mask)

        # Important, the bias of the last linear might be non-zero
        if node_mask is not None:
            h = h * node_mask
        return h, x


class EGNN(nn.Module):
    def __init__(self, in_node_nf, in_edge_nf, hidden_nf, device='cpu', act_fn=nn.SiLU(), n_layers=3, attention=False,
                 norm_diff=True, out_node_nf=None, tanh=False, coords_range=15, norm_constant=1, inv_sublayers=2,
                 sin_embedding=False, normalization_factor=100, aggregation_method='sum'):
        super(EGNN, self).__init__()
        if out_node_nf is None:
            out_node_nf = in_node_nf
        self.hidden_nf = hidden_nf
        self.device = device
        self.n_layers = n_layers
        self.coords_range_layer = float(coords_range/n_layers) if n_layers > 0 else float(coords_range)
        self.norm_diff = norm_diff
        self.normalization_factor = normalization_factor
        self.aggregation_method = aggregation_method

        if sin_embedding:
            self.sin_embedding = SinusoidsEmbeddingNew()
            edge_feat_nf = self.sin_embedding.dim * 2
        else:
            self.sin_embedding = None
            edge_feat_nf = 2

        self.embedding = nn.Linear(in_node_nf, self.hidden_nf)
        self.embedding_out = nn.Linear(self.hidden_nf, out_node_nf)
        for i in range(0, n_layers):
            self.add_module("e_block_%d" % i, EquivariantBlock(hidden_nf, edge_feat_nf=edge_feat_nf, device=device,
                                                               act_fn=act_fn, n_layers=inv_sublayers,
                                                               attention=attention, norm_diff=norm_diff, tanh=tanh,
                                                               coords_range=coords_range, norm_constant=norm_constant,
                                                               sin_embedding=self.sin_embedding,
                                                               normalization_factor=self.normalization_factor,
                                                               aggregation_method=self.aggregation_method))
        self.to(self.device)

    def forward(self, h, x, edge_index, node_mask=None, edge_mask=None):
        # Edit Emiel: Remove velocity as input
        distances, _ = coord2diff(x, edge_index)
        if self.sin_embedding is not None:
            distances = self.sin_embedding(distances)
        h = self.embedding(h)
        for i in range(0, self.n_layers):
            h, x = self._modules["e_block_%d" % i](h, x, edge_index, node_mask=node_mask, edge_mask=edge_mask, edge_attr=distances)

        # Important, the bias of the last linear might be non-zero
        h = self.embedding_out(h)
        if node_mask is not None:
            h = h * node_mask
        return h, x




def fully_connected_graph_with_self_loops(num_nodes):
    """
    Generates edge indices for a fully connected graph with self-loops.

    Args:
        num_nodes (int): Number of nodes in the graph.

    Returns:
        torch.Tensor: Edge indices of the fully connected graph with self-loops.
    """
    # Create edge indices for a fully connected graph with self-loops
    edge_index = torch.tensor([[i, j] for i in range(num_nodes) for j in range(num_nodes)])

    return edge_index.t().contiguous()




Calculating the degree for PNA

In [5]:
# Compute the maximum in-degree in the training data.
max_degree = -1
for data in filtered_dataset:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    max_degree = max(max_degree, int(d.max()))

# Compute the in-degree histogram tensor
deg = torch.zeros(max_degree + 1, dtype=torch.long)
for data in filtered_dataset:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

Defining the Autoencoder

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

from torch.nn import Linear
from torch_geometric.nn import PNA

class GraphAutoencoder(nn.Module):
    def __init__(self):
        super(GraphAutoencoder, self).__init__()

        # Define the parameters for the autoencoder

        in_edge_nf = 0  # Replace with the actual input edge feature dimension
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        act_fn = torch.nn.SiLU()
        n_layers = 4  # Replace with the desired number of layers


        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']
        self.conv1 =  PNA(
            in_channels=5,
            hidden_channels= 10,

            out_channels=6,
            num_layers=2,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            towers=1,

            post_layers=1,
            pre_layers=1
        )

        self.dconv1 = PNA(
            in_channels=6,
            hidden_channels= 10,
            out_channels=5,
            num_layers=2,
            aggregators=aggregators,
            scalers=scalers,
            deg=deg,
            towers=1,

            post_layers=1,
            pre_layers=1
        )

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)


        return x




    def decode(self, z, edge_index):
        z = self.dconv1(z, edge_index)


        return z




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= GraphAutoencoder()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = nn.MSELoss()





Training the Autoencoder


In [ ]:
import time
import pickle


for epoch in range(20):
    total_loss = 0
    start_time = time.time()

    print(epoch)
    for data in filtered_dataset:







        edge_index_1 = data.edge_index
        node_features_1 = data.x[:,:5]
        num_nodes_1 = node_features_1.size(0)

        adjacency_matrix_1 = edge_index_to_adj(edge_index_1, num_nodes_1)




        num_vectors_1 = node_features_1.size(0)
        num_upper_triangle_terms_1 = int((num_vectors_1 * (num_vectors_1 - 1)) / 2)

        pairwise_distances_1 = torch.zeros(num_upper_triangle_terms_1)

        k= 0
        # Calculate pairwise distances
        for i in range(num_vectors_1):
            for j in range(i + 1, num_vectors_1):


                pairwise_distances_1[k] = adjacency_matrix_1[i][j]

                k=k+ 1



        pairwise_distances_1 = pairwise_distances_1.view(-1, 1)
        pairwise_distances_1= torch.where(pairwise_distances_1 == 0, 1, pairwise_distances_1*0)

        column_tensor_1 = pairwise_distances_1

        num_repeats_1 = 5

        row_tensor_1 = column_tensor_1.repeat(1, num_repeats_1)


        updated_node_features_1 = torch.cat([node_features_1, row_tensor_1], dim=0)


        # Backpropagation and optimization
        optimizer.zero_grad()

        z = model.encode(node_features_1,data.edge_index)

        vectors = z


        # Initialize an empty matrix to store pairwise distances
        num_vectors = vectors.size(0)
        num_upper_triangle_terms = int((num_vectors * (num_vectors - 1)) / 2)
        matrix = torch.zeros(num_vectors+ num_upper_triangle_terms, num_vectors+ num_upper_triangle_terms)
        pairwise_distances = torch.zeros(num_upper_triangle_terms)

        k= 0
        # Calculate pairwise distances
        for i in range(num_vectors):
            for j in range(i + 1, num_vectors):
                matrix[i,num_vectors+ k] = 1
                matrix[j,num_vectors+ k] = 1
                matrix[num_vectors+ k,i] = 1
                matrix[num_vectors+ k,j] = 1


                distance = torch.norm(vectors[i] - vectors[j])

                pairwise_distances[k] = distance
                k=k+ 1


        pairwise_distances = pairwise_distances.view(-1, 1)

        column_tensor = pairwise_distances

        num_repeats = 6

        row_tensor = column_tensor.repeat(1, num_repeats)

        updated_node_features = torch.cat([z, row_tensor], dim=0)

        adjacency_matrix = matrix
        num_nodes = adjacency_matrix.size(0)

        updated_edge_index = [[], []]

        for i in range(num_nodes):
            for j in range(num_nodes):
                if adjacency_matrix[i, j] == 1:
                    updated_edge_index[0].append(i)  # Source node
                    updated_edge_index[1].append(j)  # Target node


        edge_index = torch.tensor(updated_edge_index)


        recon = model.decode(updated_node_features, edge_index)

        loss = criterion(recon, updated_node_features_1)

        loss.backward()
        optimizer.step()


        total_loss += loss.item()
    average_loss = total_loss /len( filtered_dataset)

    print(f"Epoch [{epoch + 1}/{epochs}] Loss: {average_loss:.4f}")

    end_time = time.time()
    training_time = end_time - start_time
    print(f"Total training time: {training_time:.2f} seconds")
    with open(f'EGNN_2D_model_epoch_{epoch + 1}.pkl', 'wb') as f:
        pickle.dump(model, f)

        print(f"Model saved for epoch {epoch + 1}")



Defining the Atom Autoencoder

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

from torch.nn import Linear
from torch_geometric.nn import PNAConv

class nodeAutoencoder(nn.Module):
    def __init__(self):
        super(nodeAutoencoder, self).__init__()

        # Define the parameters for the autoencoder

        in_edge_nf = 0  # Replace with the actual input edge feature dimension
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        act_fn = torch.nn.SiLU()
        n_layers = 4  # Replace with the desired number of layers


        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']



        self.conv1 = PNAConv(in_channels=5, out_channels=2,
                           aggregators=aggregators, scalers=scalers, deg=deg)





        self.dconv1 = PNAConv(in_channels=2, out_channels=5,
                           aggregators=aggregators, scalers=scalers, deg=deg)





    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index)

        return x






    def decode(self, z, edge_index):
        z = self.dconv1(z, edge_index)

        return z



# Define other necessary components and hyperparameters
epochs = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= nodeAutoencoder()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = nn.MSELoss()





Training  the Atom Autoencoder

In [ ]:

import time
import pickle

data1=filtered_dataset
for epoch in range(10):
    start_time = time.time()

    total_loss = 0


    print(epoch)
    for data in data1:

        edge_index_1 = data.edge_index.to(device)

        feat = data.x.to(device)

        num_nodes_1 = feat.size(0)

        adjacency_matrix_1 = edge_index_to_adj(edge_index_1, num_nodes_1)

        node_features_1= feat[:,:5]

        num_nodes_1 = node_features_1.size(0)

        num_vectors_1 = node_features_1.size(0)

        optimizer.zero_grad()

        z =model.encode(node_features_1,edge_index_1.to(device)) #encode

        edge_index1 = fully_connected_graph_with_self_loops(num_nodes_1)

        recon = model.decode(z.to(device), edge_index1.to(device))


        num_rows2 = recon.size(0)


        loss = criterion(recon,node_features_1)

        loss.backward()
        optimizer.step()


        total_loss += loss.item()



    average_loss = total_loss / len(data1)


    print(f"Epoch [{epoch + 1}/{epochs}] Loss: {average_loss:.4f}")
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Total training time: {training_time:.2f} seconds")
    with open(f'Atom_encoder_model_epoch_{epoch + 1}.pkl', 'wb') as f:
        pickle.dump(model, f)

        print(f"Model saved for epoch {epoch + 1}")





Load the best model from the trained models

In [ ]:
import pickle
with open('LOAD_YOUR_BEST_AUTOENCODER_MODEL_HERE', 'rb') as f:
    loaded_auto = pickle.load(f)

In [ ]:
with open('LOAD_YOUR_BEST_ATOM_AUTOENCODER_HERE', 'rb') as f:
    loaded_node = pickle.load(f)

Defining the parameters for Diffusion

In [ ]:
def cosine_beta_schedule(timesteps, s=0.008):

    steps = timesteps + 1
    x = torch.linspace(0, timesteps, steps)
    alphas_cumprod = torch.cos(((x / timesteps) + s) / (1 + s) * torch.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    return torch.clip(betas, 0.0001, 0.9999)

def linear_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)

def quadratic_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start**0.5, beta_end**0.5, timesteps) ** 2

def sigmoid_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    betas = torch.linspace(-6, 6, timesteps)
    return torch.sigmoid(betas) * (beta_end - beta_start) + beta_start


In [ ]:
timesteps = 50

# define beta schedule
betas = linear_beta_schedule(timesteps=timesteps)


# define alphas
alphas = 1. - betas

alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)

# calculations for diffusion q(x_t | x_{t-1}) and others
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)

# calculations for posterior q(x_{t-1} | x_t, x_0)
posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

def extract(a, t, x_shape):
    batch_size = t.shape[0]
    out = a.gather(-1, t.cpu())
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)

In [ ]:
# forward diffusion
def q_sample(x_start, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x_start)

    sqrt_alphas_cumprod_t = extract(sqrt_alphas_cumprod, t, x_start.shape)
    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, x_start.shape
    )

    return sqrt_alphas_cumprod_t.to(device) * x_start.to(device) + sqrt_one_minus_alphas_cumprod_t.to(device) * noise.to(device), noise.to(device)

In [ ]:
def get_noisy_image(x_start, t):

  x_noisy,tar_noise = q_sample(x_start, t=t)
  return x_noisy,tar_noise

Defining the restoration Model

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import add_self_loops

# Assume you have input features, edge_index, and edge_attr tensors
in_node_nf = 1 # Replace with the actual input node feature dimension
in_edge_nf = 0  # Replace with the actual input edge feature dimension
hidden_nf = 64 # Replace with the desired hidden dimension
device = 'cuda' if torch.cuda.is_available() else 'cpu'
act_fn = torch.nn.SiLU()
n_layers = 7  # Replace with the desired number of layers

# Create a simple graph data


# Instantiate the EGNN model
model = EGNN(
    in_node_nf=in_node_nf,
    in_edge_nf=in_edge_nf,
    hidden_nf=hidden_nf,
    device=device,
    act_fn=act_fn,
    n_layers=n_layers
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Training the Restoration Model


In [ ]:
import time
epochs =20
data1= filtered_dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for epoch in range(epochs):
    total_loss = 0

    start_time = time.time()




    print(epoch)

    for data in data1:





        edge_index_1 = data.edge_index
        node_features_1 = data.x[:,:5]

        optimizer.zero_grad()
        node= node_features_1[:,:5]

        z = loaded_auto.encode(node_features_1, edge_index_1)
        y= loaded_node.encode(node,edge_index_1.to(device))
        x= torch.cat([z,y ], dim=1)
        v=x.to(device)

        random_timestep = torch.randint(1, 50, size=(1,))

        t = torch.tensor([random_timestep])
        h_time = torch.empty_like(x[:, 0:1]).fill_(t.item()/50)

        original_tensor = x

        ans,tar_noise= get_noisy_image(original_tensor, t)

        num_nodes1=ans.shape[0]

        edge_index1 = fully_connected_graph_with_self_loops(num_nodes1).to(device)

        g,output = model(h_time,ans, edge_index1)
        new_z= output-ans

        loss= criterion(new_z, tar_noise)
        loss.backward()
        optimizer.step()


        total_loss += loss.item()
    average_loss = total_loss /len( filtered_dataset)

    print(f"Epoch [{epoch + 1}/{epochs}] Loss: {average_loss:.4f}")
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Total training time: {training_time:.2f} seconds")
    with open(f'EGNN_2D_diffusion_model_epoch_{epoch + 1}.pkl', 'wb') as f:
        pickle.dump(model, f)

        print(f"Model saved for epoch {epoch + 1}")











Training the restoration model

In [ ]:
with open('LOAD_YOUR_BEST_DIFFUSION_MODEL_HERE', 'rb') as f:
    loaded_deno = pickle.load(f)

Importing elements for Sampling

In [ ]:
import math
import operator
from itertools import chain, product
from functools import partial
from pathlib import Path
from typing import Any, Optional, Callable, Tuple, Dict, Sequence, NamedTuple

import numpy as np

from tqdm import tqdm

import torch
import torch.nn as nn
from torch import Tensor, LongTensor

In [ ]:
#come here
@torch.no_grad()
def p_sample(model, data, t, t_index):
    t1 = t_index
    h_time = torch.empty_like(data[:, 0:1]).fill_(t1/100)
    # Concatenate h and h_time along dimension 1
    ans = data


    num_nodes1=ans.shape[0]
    edge_index1 = fully_connected_graph(num_nodes1).to(device)
    s= torch.cat([ans,h_time ], dim=1)

    g,output = loaded_deno(h_time,ans, edge_index1)
    new_z= output-ans




    betas_t = extract(betas, t, ans.shape)

    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t,ans.shape
    )
    sqrt_recip_alphas_t = extract(sqrt_recip_alphas, t, ans.shape)

    model_mean = sqrt_recip_alphas_t * (
        ans - betas_t * new_z / sqrt_one_minus_alphas_cumprod_t
    )

    if t_index == 0:
        return model_mean
    else:
        posterior_variance_t = extract(posterior_variance, t, ans.shape)
        noise = torch.randn_like(ans)

        return model_mean + torch.sqrt(posterior_variance_t) * noise

@torch.no_grad()
def p_sample_loop(model,data):
    device = next(model.parameters()).device

    b = 1

    t_init= torch.tensor([49])
    imgs = []
    features,tar= get_noisy_image(data, t_init)


    for i in tqdm(reversed(range(0, 50)), desc='sampling loop time step', total=50):


        data = p_sample(model, data, torch.full((b,), i, device=device, dtype=torch.long), i)

        imgs.append(data.cpu().numpy())

    return imgs

@torch.no_grad()
def sample(model, data):
    return p_sample_loop(model, data)

Defining the Bond Type Predictor Model

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv, MessagePassing
from torch_geometric.data import Data

class BondTypePredictor(nn.Module):
    def __init__(self, num_node_features, hidden_dim, num_classes):
        """
        Initialize the BondTypePredictor model.

        Args:
            num_node_features (int): Number of input features for each node.
            hidden_dim (int): Dimension of hidden layers in GNN.
            num_classes (int): Number of bond types to predict.
        """
        super(BondTypePredictor, self).__init__()

        # GNN Layers to process the molecular graph
        self.conv1 = GCNConv(num_node_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)


        self.edge_classifier = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x, edge_index):
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Node feature matrix of shape [num_nodes, num_node_features].
            edge_index (torch.Tensor): Edge index matrix of shape [2, num_edges].

        Returns:
            torch.Tensor: Predicted bond types for each edge of shape [num_edges, num_classes].
        """

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)

        row, col = edge_index
        edge_representation = torch.cat([x[row], x[col]], dim=1)


        bond_type_logits = self.edge_classifier(edge_representation)

        return bond_type_logits


In [ ]:
with open('LOAD_YOUR_BEST_BOND_TYPE_PREDICTION_MODEL', 'rb') as f:
    edge_type= pickle.load(f)

The Sampling process

In [ ]:
import random
smiles_list = []
for i in range(100):


    random_number = random.randint(2, 8)

    real_num=random_number

    num_vectors=real_num
    num_vectors = real_num

    edge_index1 =fully_connected_graph_with_self_loops(num_vectors)


    data2=torch.randn(num_vectors,8).to(device)

    samples = sample(model, data2)
    final=samples[49]
    final=torch.tensor(final)
    latent= final[:, :6]

    z= latent

    vectors = z

    num_vectors = vectors.size(0)
    num_upper_triangle_terms = int((num_vectors * (num_vectors - 1)) / 2)
    matrix = torch.zeros(num_vectors+ num_upper_triangle_terms, num_vectors+ num_upper_triangle_terms)
    pairwise_distances = torch.zeros(num_upper_triangle_terms)

    k= 0
    # Calculate pairwise distances
    for i in range(num_vectors):
        for j in range(i + 1, num_vectors):
            matrix[i,num_vectors+ k] = 1
            matrix[j,num_vectors+ k] = 1
            matrix[num_vectors+ k,i] = 1
            matrix[num_vectors+ k,j] = 1


            distance = torch.norm(vectors[i] - vectors[j])

            pairwise_distances[k] = distance
            k=k+ 1




    pairwise_distances = pairwise_distances.view(-1, 1)


    column_tensor = pairwise_distances

    num_repeats = 6

    row_tensor = column_tensor.repeat(1, num_repeats)


    updated_node_features = torch.cat([z, row_tensor], dim=0)

    adjacency_matrix = matrix

    num_nodes = adjacency_matrix.size(0)

    updated_edge_index = [[], []]

    for i in range(num_nodes):
        for j in range(num_nodes):
            if adjacency_matrix[i, j] == 1:
                updated_edge_index[0].append(i)  # Source node
                updated_edge_index[1].append(j)  # Target node

    edge_index = torch.tensor(updated_edge_index)

    recon = loaded_auto.decode(updated_node_features, edge_index)
    node = final[:, -2:]
    node_out = loaded_node.decode(node.to(device), edge_index1.to(device))

    des= recon[:num_vectors]

    ans= node_out

    max_values, _ = torch.max(ans, dim=1, keepdim=True)

    ans_binary = torch.where(ans == max_values, torch.tensor(1.0), torch.tensor(0.0))

    row_values = []

    for row in ans_binary:

        binary_str = ''.join([str(int(x)) for x in row.tolist()])

        if binary_str == '01000':
            row_values.append(6)
        elif binary_str == '10000':
            row_values.append(1)
        elif binary_str == '00100':
            row_values.append(7)
        elif binary_str == '00010':
            row_values.append(8)
        elif binary_str == '00001':
            row_values.append(9)
        else:

            row_values.append(0)


    new_tensor = torch.tensor(row_values)

    augmented_features_matrix = recon

    node_features_matrix = augmented_features_matrix[:num_vectors, :]
    edge_features_matrix = augmented_features_matrix[num_vectors:, :]
    row_averages = torch.mean(edge_features_matrix, dim=1)
    row_averages = (row_averages > 0).float()

    adjacency_matrix_dec = torch.zeros((num_vectors, num_vectors), dtype=torch.int)

    k= 0
    # Calculate pairwise distances
    for i in range(num_vectors):
        for j in range(i + 1, num_vectors):
            adjacency_matrix_dec[i][j]=row_averages[k]
            adjacency_matrix_dec[j][i]=row_averages[k]



            k=k+ 1

    edge_list = [(i, j) for i in range(adjacency_matrix_dec.size(0)) for j in range(adjacency_matrix_dec.size(1)) if adjacency_matrix_dec[i][j] != 0]

    edge_index = torch.tensor(edge_list).t().contiguous()
    if edge_index.dim() < 2 or edge_index.size(1) == 0:

        edge_attr = torch.empty(0, dtype=torch.long)

    else:
    # If edges are present, run the bond type prediction
        bond_type_logits = edge_type(ans_binary[:, :5], edge_index)

    # Get the predicted bond type for each edge
        edge_attr = torch.argmax(bond_type_logits, dim=1)  # Shape: [num_edges]


    import torch
    from rdkit import Chem

    # Assuming 'data' is your graph representation with 'data.x' for node features and 'data.edge_index' for edge connections
    # You need to convert your graph to an RDKit Mol object first
    mol = Chem.RWMol()

    # Define the mapping from integer bond type to RDKit bond type
    bond_type_mapping = {
        1: Chem.BondType.SINGLE,
        2: Chem.BondType.DOUBLE,
        3: Chem.BondType.TRIPLE,
        4: Chem.BondType.AROMATIC
    }

    # Keep track of the bonds already added to avoid duplicates
    added_bonds = set()

    # Add atoms to the molecule based on node features (data.x)
    for atom_type in new_tensor:
        atom = Chem.Atom(atom_type.item())
        mol.AddAtom(atom)

    # Add bonds to the molecule based on edge connections (edge_index) and edge types (edge_attr)
    for (i, j), bond_type in zip(edge_index.t().tolist(), edge_attr.tolist()):
        # Check if the bond already exists
        if (i, j) not in added_bonds:
            # Get the appropriate RDKit bond type based on bond_type
            rdkit_bond_type = bond_type_mapping.get(bond_type, Chem.BondType.SINGLE)  # Default to SINGLE if bond_type is unknown

            # Add the bond to the molecule with the specified bond type
            mol.AddBond(i, j, rdkit_bond_type)

            # Mark this bond as added to avoid duplicates
            added_bonds.add((i, j))
            added_bonds.add((j, i))  # Assuming the graph is undirected

    # Convert the RDKit Mol object to a SMILES string
    smiles = Chem.MolToSmiles(mol)
    generated_smiles = smiles
    smiles_list.append(generated_smiles)



    # Check if the molecule is valid (i.e., if it has correct atom types, valence, etc.)
    def is_valid_molecule(mol):
        if mol is None:
            return False
        return Chem.SanitizeMol(mol) == Chem.SanitizeFlags.SANITIZE_NONE

    mol = Chem.MolFromSmiles(smiles)
    valid = is_valid_molecule(mol)




Computing the Validity, Uniqueness and Novelty


In [ ]:
from rdkit import Chem


def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None


def get_largest_fragment(mol):
    mol_frags = Chem.rdmolops.GetMolFrags(mol, asMols=True)
    largest_frag = max(mol_frags, key=lambda m: m.GetNumAtoms(), default=None)
    return Chem.MolToSmiles(largest_frag) if largest_frag else None

# Function to check and count the number of valid SMILES
def count_valid_smiles(smiles_list):
    valid_smiles = []
    fragmented_smiles = []

    for smiles in smiles_list:
        if is_valid_smiles(smiles):
            valid_smiles.append(smiles)
            mol = Chem.MolFromSmiles(smiles)
            largest_fragment_smile = get_largest_fragment(mol)
            if largest_fragment_smile:
                fragmented_smiles.append(largest_fragment_smile)


    print(f"VALID SMILES : {fragmented_smiles}")
    print(f"Number of VALID SMILES: {len(fragmented_smiles)}")

    return fragmented_smiles


def find_unique_smiles(generated_smiles):
    smiles_count = {}

    # Count occurrences of each SMILES
    for smiles in generated_smiles:
        if smiles in smiles_count:
            smiles_count[smiles] += 1
        else:
            smiles_count[smiles] = 1

    # Collect SMILES that appear only once
    unique_smiles = [smiles for smiles, count in smiles_count.items() if count == 1]

    return unique_smiles

# Function to find unique SMILES not in the dataset
def find_novel_smiles(generated_smiles, dataset):
    novel_smiles = []
    for smiles in generated_smiles:
        if smiles not in dataset:  # Check if the SMILES is not in the dataset
            novel_smiles.append(smiles)
    return novel_smiles


fragmented_smiles = count_valid_smiles(smiles_list)

# Find and print the unique SMILES in the fragmented list
unique_smiles = find_unique_smiles(fragmented_smiles)
print(f"Number of unique SMILES generated: {len(unique_smiles)}")
print(f"Percetage of unique SMILES generated: {len(unique_smiles)/len(fragmented_smiles)}")


# Find and print the novel SMILES not in the dataset
novel_smiles = find_novel_smiles(unique_smiles, modified_smiles)
print(f"Novel SMILES not in dataset: {novel_smiles}")
print(f"Number of novel SMILES not in dataset: {len(novel_smiles)}")
print(f"novelty: {len(novel_smiles)/len(unique_smiles)}")

